In [2]:
import torch
torch.manual_seed(7)

In [13]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
loss.backward()
print(w.grad)
print(b.grad)
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

z = torch.matmul(x, w) + b
print(z.requires_grad)
z_det = z.detach()
print(z_det.requires_grad)

tensor([[0.0227, 0.0738, 0.0327],
        [0.0227, 0.0738, 0.0327],
        [0.0227, 0.0738, 0.0327],
        [0.0227, 0.0738, 0.0327],
        [0.0227, 0.0738, 0.0327]])
tensor([0.0227, 0.0738, 0.0327])
True
False
True
False


torch.autograd.backward(tensors, grad_tensors=None, retain_graph=None, create_graph=False)

功能：自动求取梯度
* tensors：用于求导的张量，如 loss
* retain_graph：保存计算图
* create_graph：创建导数计算图，用于高阶求导
* grad_tensors：多梯度权重

In [2]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
c = torch.mul(a, b)

c.backward(retain_graph=True)

In [3]:
# 2 + 3
w.grad

tensor([5.])

In [6]:
# 2
x.grad

tensor([2.])

torch.autograd.grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False)

功能：求取梯度
* outputs：用于求导的张量，如 loss
* inputs：需要梯度的张量
* create_graph：创建导数计算图，用于高阶求导
* retain_graph：保存计算图
* grad_outputs：多梯度权重

Tips

* 梯度不自动清零
* 依赖于叶子节点的节点，requires_grad默认为True
* 叶子节点不可执行in-place

autograd包是PyTorch中所有神经网络的核心。

* torch.Tensor是包的核心类。如果将其属性.requires_grad设置为True，则会开始跟踪针对tensor的所有操作。完成计算后，你可以调用.backward()来自动计算所有梯度。该张量的梯度将累积到.grad属性中。

* 要停止tensor历史记录的跟踪，你可以调用.detach()，它将其与计算历史记录分离，并防止将来的计算被跟踪。

* 要停止跟踪历史记录（和使用内存），你还可以将代码块使用with torch.no_grad()：包装起来。在评估模型时，这是特别有用，因为模型在训练阶段具有requires_grad=True的可训练参数有利于调参，但在评估阶段我们不需要梯度。

* 还有一个类对于autograd实现非常重要那就是Function。Tensor和Function互相连接并构建一个非循环图，它保存整个完整的计算过程的历史信息。每个张量都有一个.grad_fn属性保存着创建了张量的Function的引用（如果用户自己创建张量，则grad_fn是None）。

* 如果你想计算导数，你可以调用Tensor.backward()。如果Tensor是标量（即它包含一个元素数据），则不需要指定backward()的任何参数，但是如果它有更多元素，则需要指定一个gradient参数来指定张量的形状。

In [22]:
x = torch.randn(3, requires_grad=True)
y = x * 2

while y.data.norm() < 1000:
    y = y * 2
    
y

tensor([-364.1600, 1261.9630,  141.4888], grad_fn=<MulBackward0>)

现在在这种情况下，y不再是一个标量。torch.autograd不能够直接计算整个雅可比，但是如果我们只想要雅可比向量积，只需要简单地传递向量给backward作为参数。

In [23]:
y.backward(torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float))

x.grad

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])

你可以通过将代码包裹在with torch.no_grad()，来停止对从跟踪历史中的.requires_grad=True的张量自动求导。

In [26]:
print(x.requires_grad)
print((x ** 2).requires_grad)
with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


求y = x^2的一阶偏导和二阶偏导。

In [27]:
x = torch.tensor([3.], requires_grad=True)
y = torch.pow(x, 2)

grad_1 = torch.autograd.grad(y, x, create_graph=True)
grad_2 = torch.autograd.grad(grad_1[0], x)

In [28]:
grad_1

(tensor([6.], grad_fn=<MulBackward0>),)

In [29]:
grad_2

(tensor([2.]),)